In [1]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import csv

In [5]:
# load dataset and drugbank

with open("../data/drd3_chembl_cmpds.csv", 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=";")
    drd3_ligands = [Chem.MolFromSmiles(m['Smiles']) for m in reader]

suppl = Chem.SDMolSupplier("../data/drugbank.sdf")
drugs = [m for m in suppl if m]  

[12:34:02] Explicit valence for atom # 7 N, 4, is greater than permitted
[12:34:02] ERROR: Could not sanitize molecule ending on line 20009
[12:34:02] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
[12:34:03] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6 7 8 9 10
[12:34:03] ERROR: Could not sanitize molecule ending on line 250947
[12:34:03] ERROR: Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6 7 8 9 10
[12:34:03] Explicit valence for atom # 17 O, 3, is greater than permitted
[12:34:03] ERROR: Could not sanitize molecule ending on line 258130
[12:34:03] ERROR: Explicit valence for atom # 17 O, 3, is greater than permitted
[12:34:03] Can't kekulize mol.  Unkekulized atoms: 57 58 59 60 61 62 63 64 65
[12:34:03] ERROR: Could not sanitize molecule ending on line 261581
[12:34:03] ERROR: Can't kekulize mol.  Unkekulized atoms: 57 58 59 60 61 62 63 64 65
[12:34:03] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 7 8 9 10 11 12 13 14 15 16 17
[12:34:03] ERROR: Coul

assemble a structural key from several substructures that were introduced in the previous excercise 5:

In [4]:
ethanol_pattern = Chem.MolFromSmarts('CCO')
propanol_pattern = Chem.MolFromSmarts('CCCO')
cooh_pattern = Chem.MolFromSmarts('C(=O)[O;h1]')
salicylic_acid_pattern = Chem.MolFromSmarts('c1ccc(c(c1)C(=O)O)O')

we will search all 4 substructures in all molecules and than we produce a binary vector of 4 boolean values

In [10]:
custom_key = [ethanol_pattern, propanol_pattern, cooh_pattern, salicylic_acid_pattern]
drd3_ligand_keys = [[m.HasSubstructMatch(p) for p in custom_key] for m in drd3_ligands]
len(drd3_ligand_keys), drd3_ligand_keys

(5938,
 [[True, True, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [True, True, False, False],
  [True, True, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [True, True, False, False],
  [True, False, True, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [True, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [True, True, False, False],
  [False, False, False, False],
  [True, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [False, False, False, False],
  [True, False, False, False],
  [False, False,

As shown above, every molecule in our ligand set is represented by four bits that correspond to the presence of our four custom-defined substructures. This is of course a toy example, and structural keys are often hundreds of bits long. However, you don't have to manually define hundreds of substructures everytime you want to use structural keys. There are several already established sets of substructure features that you can use if you do not have a specific substructures in mind, and want a general characterization of your structure set. You can always extend those generic keys by whatever explicit groups you want to focus on.

# MACCS key
is an often-used structural key consisting of 166 defined [substructures of interest](https://github.com/openbabel/openbabel/blob/master/data/MACCS.txt), also implemented in RDKit. Let's make use of it to characterize our known ligands and the DrugBank database:

In [15]:
from rdkit.Chem import MACCSkeys

In [16]:
drd3_maccs = [MACCSkeys.GenMACCSKeys(m) for m in drd3_ligands]
drugbank_macss = [MACCSkeys.GenMACCSKeys(m) for m in drugs]
drd3_maccs[0]

In [18]:
# as with the custom substructure keys, each molecule is represented by a vector of
# binary values. In the RDkit implementation, the binary vector is implemented as an RDkit
# object has some nice inbuilt binary vector operations.

drd3_maccs[0].GetNumBits() # get the length of the vector

167

In [19]:
list(drugbank_macss[0].GetOnBits()) # get the indices of the bits that are set to 1 within the fingerprint

[25,
 43,
 53,
 54,
 74,
 75,
 77,
 78,
 79,
 80,
 82,
 83,
 84,
 85,
 90,
 91,
 92,
 95,
 96,
 97,
 100,
 104,
 106,
 110,
 111,
 113,
 114,
 115,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 125,
 127,
 128,
 129,
 131,
 132,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165]

In [20]:
drd3_maccs[0].GetBit(25), drd3_maccs[0].GetBit(26) # get the value of a specific bit

(False, False)

In [22]:
drd3_maccs[0].ToBitString() # get the full binary vector as a string

'00000000000000000000000000001000000000000000000000000000010000100000000010110001000101100010110111101100110001110100011011110010110001010110111011111101111111111111110'

In [24]:
# compare the relative amounts of substructures within our set of ligands and the DrugBank contents:

drd3_ligands_maccs_sums = [0]*drd3_maccs[0].GetNumBits() # create a vector of zeros
for m in drd3_maccs:
    for onbit in m.GetOnBits():
        drd3_ligands_maccs_sums[onbit] += 1

drd3_ligands_maccs_sums

[0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 36,
 0,
 0,
 117,
 0,
 8,
 2,
 2,
 3,
 68,
 1,
 604,
 6,
 3,
 327,
 124,
 97,
 230,
 64,
 69,
 244,
 2,
 14,
 5,
 406,
 422,
 28,
 1,
 622,
 389,
 1379,
 85,
 116,
 216,
 1416,
 117,
 4,
 30,
 234,
 741,
 87,
 117,
 208,
 607,
 1009,
 267,
 535,
 589,
 60,
 1621,
 596,
 1050,
 589,
 624,
 4345,
 60,
 976,
 3152,
 805,
 665,
 12,
 301,
 212,
 349,
 1035,
 596,
 439,
 5192,
 233,
 2146,
 138,
 3943,
 2334,
 1553,
 1818,
 3695,
 254,
 5445,
 5378,
 2225,
 1668,
 805,
 2738,
 2898,
 3041,
 2559,
 1674,
 2391,
 3846,
 2307,
 4850,
 475,
 5621,
 3877,
 947,
 1582,
 2933,
 4154,
 1976,
 2752,
 1545,
 1578,
 3564,
 5550,
 1726,
 2916,
 822,
 1068,
 940,
 3864,
 5551,
 204,
 4976,
 5566,
 5547,
 774,
 1893,
 5285,
 2303,
 3107,
 5476,
 5536,
 902,
 1556,
 2236,
 3988,
 2981,
 3364,
 1358,
 5676,
 5437,
 1297,
 1013,
 730,
 5379,
 3107,
 4217,
 5668,
 2100,
 5564,
 5609,
 1835,
 3402,
 3571,
 3808,
 5733,
 3631,
 5487,
 5605,
 3551,
 5820,
 3677,
 3836,
 5845,
 5

In [25]:
drugbank_maccs_sums = [0]*drugbank_macss[0].GetNumBits() # create a vector of zeros
for m in drugbank_macss:
    for onbit in m.GetOnBits():
        drugbank_maccs_sums[onbit] += 1

drugbank_maccs_sums

[0,
 0,
 0,
 30,
 0,
 1,
 11,
 25,
 99,
 46,
 26,
 117,
 30,
 60,
 33,
 11,
 52,
 77,
 68,
 282,
 7,
 21,
 181,
 212,
 435,
 565,
 227,
 96,
 149,
 727,
 126,
 21,
 431,
 481,
 155,
 44,
 625,
 521,
 1313,
 153,
 167,
 166,
 698,
 1199,
 270,
 249,
 194,
 455,
 870,
 716,
 649,
 601,
 553,
 1896,
 2247,
 663,
 178,
 1634,
 669,
 604,
 685,
 695,
 1753,
 207,
 625,
 2511,
 1118,
 783,
 133,
 1552,
 295,
 526,
 2197,
 755,
 1187,
 2092,
 801,
 2249,
 706,
 2264,
 2300,
 1229,
 2080,
 2784,
 2110,
 2498,
 1685,
 1310,
 1597,
 2478,
 3534,
 3343,
 2826,
 1653,
 1484,
 3192,
 3101,
 2787,
 3060,
 1149,
 2845,
 2779,
 1935,
 786,
 3210,
 2943,
 2669,
 1534,
 1735,
 2376,
 3114,
 3377,
 2729,
 2293,
 1100,
 1831,
 1899,
 3355,
 3350,
 927,
 3227,
 3766,
 2976,
 2794,
 2467,
 3452,
 2551,
 3825,
 2692,
 2853,
 1795,
 4639,
 3558,
 2718,
 1637,
 2199,
 3268,
 4496,
 2582,
 4292,
 3266,
 1355,
 4173,
 3825,
 2806,
 3702,
 4379,
 3399,
 3797,
 2510,
 3943,
 4408,
 3967,
 4554,
 4674,
 4855,
 5258

In [26]:
# because the dataset sizes are different, we divide the raw incidence counts by the total
# size, thus getting a number between 0 and 1 that represents the relative frequency of
# the substructure within the dataset for both the ligands and the DrugBank set.

drd3_ligands_maccs_scaled = [x/len(drd3_maccs) for x in drd3_ligands_maccs_sums]
drd3_ligands_maccs_scaled

[0.0,
 0.0,
 0.0,
 0.0003368137420006736,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006062647356012125,
 0.0,
 0.0,
 0.019703603907039405,
 0.0,
 0.0013472549680026945,
 0.0003368137420006736,
 0.0003368137420006736,
 0.0005052206130010104,
 0.011451667228022903,
 0.0001684068710003368,
 0.10171775008420343,
 0.0010104412260020209,
 0.0005052206130010104,
 0.05506904681711014,
 0.020882452004041766,
 0.01633546648703267,
 0.03873358033007747,
 0.010778039744021556,
 0.01162007409902324,
 0.04109127652408218,
 0.0003368137420006736,
 0.0023576961940047153,
 0.000842034355001684,
 0.06837318962613674,
 0.07106769956214214,
 0.004715392388009431,
 0.0001684068710003368,
 0.1047490737622095,
 0.06551027281913102,
 0.23223307510946448,
 0.01431458403502863,
 0.01953519703603907,
 0.036375884136072754,
 0.23846412933647693,
 0.019703603907039405,
 0.0006736274840013472,
 0.005052206130010104,
 0.03940720781407881,
 0.12478949141124958,
 0.014651397777029302,
 0.019703603907039405,
 0.035028629168070056,
 0

In [28]:
drugbank_maccs_scaled = [x/len(drugbank_macss) for x in drugbank_maccs_sums]
drugbank_maccs_scaled

[0.0,
 0.0,
 0.0,
 0.004215259238443164,
 0.0,
 0.0001405086412814388,
 0.0015455950540958269,
 0.00351271603203597,
 0.013910355486862442,
 0.006463397498946185,
 0.003653224673317409,
 0.01643951102992834,
 0.004215259238443164,
 0.008430518476886329,
 0.00463678516228748,
 0.0015455950540958269,
 0.007306449346634818,
 0.010819165378670788,
 0.00955458760713784,
 0.039623436841365746,
 0.0009835604889700718,
 0.002950681466910215,
 0.025432064071940423,
 0.02978783195166503,
 0.06112125895742588,
 0.07938738232401292,
 0.03189546157088661,
 0.013488829563018126,
 0.020935787550934382,
 0.10214978221160602,
 0.01770408880146129,
 0.002950681466910215,
 0.060559224392300125,
 0.06758465645637207,
 0.021778839398623014,
 0.0061823802163833074,
 0.08781790080089925,
 0.07320500210762962,
 0.18448784600252915,
 0.021497822116060137,
 0.023464943094000282,
 0.023324434452718843,
 0.0980750316144443,
 0.16846986089644514,
 0.037937333145988475,
 0.034986651679078266,
 0.02725867640859913,


In [29]:
# we have the occurrence ratio of each MACCS substructure within the ligands and the DrugBank set.
# now substract the two and have a look at the differences: 

drd3_drugbank_differences = [(i, a_b[0] - a_b[1])
                             for i, a_b in enumerate(zip(drd3_ligands_maccs_scaled, drugbank_maccs_scaled))]
drd3_drugbank_differences

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, -0.0038784454964424907),
 (4, 0.0),
 (5, -0.0001405086412814388),
 (6, -0.0015455950540958269),
 (7, -0.00351271603203597),
 (8, -0.007847708130850317),
 (9, -0.006463397498946185),
 (10, -0.003653224673317409),
 (11, 0.003264092877111064),
 (12, -0.004215259238443164),
 (13, -0.007083263508883634),
 (14, -0.004299971420286807),
 (15, -0.0012087813120951532),
 (16, -0.006801228733633808),
 (17, 0.0006325018493521153),
 (18, -0.009386180736137503),
 (19, 0.062094313242837686),
 (20, 2.6880737031949103e-05),
 (21, -0.0024454608539092046),
 (22, 0.029636982745169717),
 (23, -0.008905379947623263),
 (24, -0.04478579247039321),
 (25, -0.04065380199393545),
 (26, -0.021117421826865053),
 (27, -0.0018687554639948856),
 (28, 0.0201554889731478),
 (29, -0.10181296846960534),
 (30, -0.015346392607456575),
 (31, -0.002108647111908531),
 (32, 0.007813965233836616),
 (33, 0.0034830431057700645),
 (34, -0.017063447010613582),
 (35, -0.006013973345382971),
 (36, 

In [31]:
# sort the bits by the difference in MACCS incidence between our ligand set and the DrugBank database

drd3_drugbank_differences.sort(key=lambda x: x[1])
drd3_drugbank_differences

[(131, -0.38977849562807054),
 (139, -0.38463937669249854),
 (140, -0.288305062101838),
 (123, -0.26223422558607934),
 (146, -0.2616329110707132),
 (84, -0.25369788786975034),
 (136, -0.23048570888928463),
 (54, -0.22562524097421283),
 (53, -0.22143974931251803),
 (89, -0.21261288194013422),
 (159, -0.17871650916905257),
 (69, -0.16737894309769166),
 (43, -0.14876625698940574),
 (157, -0.14064112829432773),
 (72, -0.13439637340997246),
 (132, -0.12337198212260614),
 (102, -0.11240291404226513),
 (116, -0.10852345105313568),
 (48, -0.10759112013782246),
 (29, -0.10181296846960534),
 (130, -0.10031001345787888),
 (119, -0.09589650878382505),
 (74, -0.09285314083192),
 (112, -0.09277782271046514),
 (99, -0.08145116510721322),
 (49, -0.08090058325047078),
 (115, -0.07741278395795473),
 (78, -0.07595895254664932),
 (76, -0.073308620723354),
 (109, -0.06810248924616713),
 (141, -0.06745219310610372),
 (50, -0.056161479023583726),
 (95, -0.045842754408547404),
 (154, -0.04525204074722211),
 (

In [32]:
# we can determine MACCS bits that are least prevalent in our ligand set and most prevalent in the DrugBank set.
# these bits correspond to structural patterns in the MACCS key definition